## Data aggregation และ Grouping

เป็นขั้นตอนสำคัญสำหรับ data analysis workflow ซึ่งเกิดหลังจากการ loading, preparing และ merging ข้อมูล

จากนั้นจึงทำการ grouping ตามค่าสถิติต่าง ๆ การทำ pivot table สำหรับการทำรายงาน และ visualization ออกมา

โดยที่ Pandas library ได้เตรียม groupby interface ไว้ให้ใช้งาน

(เทียบได้กับ aggregation function และ group by ใน SQL )

## ขั้นตอนการทำงานของ GroupBy

![alt text](pyda_0901.png "Flow")

In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.DataFrame( { 'key1': ['a','b','b','b','a'],
                       'key2': ['one','two','one','two','one'],
                       'data1': np.random.randn(5),
                       'data2': np.random.randn(5)
                     } )
data

,data1,data2,key1,key2
0,0.453406,-0.112818,a,one
1,0.167646,-0.868377,b,two
2,-0.418887,1.629293,b,one
3,1.057466,-0.001176,b,two
4,-0.368687,0.811231,a,one


### ถ้าต้องการหาค่า mean ของข้อมูล data1 โดยแบ่งกลุ่มจาก key1
สามารถทำได้หลายวิธี ยกตัวอย่างเช่น

* https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.groupby.html
* https://pandas.pydata.org/pandas-docs/stable/api.html#id32

### ทำการ groupby ผ่าน Series

In [4]:
group_data = data['data1'].groupby(data['key1'])
group_data

In [6]:
group_data.mean()

key1
a    0.042359
b    0.268742
Name: data1, dtype: float64

สามารถทำการแบ่งกลุ่มด้วย key1 และ key2

In [10]:
means = data['data1'].groupby([data['key1'], data['key2']]).mean()
means

key1  key2
a     one     0.042359
b     one    -0.418887
      two     0.612556
Name: data1, dtype: float64

In [11]:
type(means)

pandas.core.series.Series

In [13]:
# Convert Series to DataFrame
means.unstack()

key2,one,two
key1,,
a,0.042359,NaN
b,-0.418887,0.612556


### ทำการ groupby ผ่าน DataFrame

https://pandas.pydata.org/pandas-docs/stable/api.html#id32

In [20]:
data

,data1,data2,key1,key2
0,0.453406,-0.112818,a,one
1,0.167646,-0.868377,b,two
2,-0.418887,1.629293,b,one
3,1.057466,-0.001176,b,two
4,-0.368687,0.811231,a,one


In [14]:
data.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.042359  0.349206
b    one  -0.418887  1.629293
     two   0.612556 -0.434777

In [16]:
data.groupby(['key1', 'key2'])['data1'].mean()

key1  key2
a     one     0.042359
b     one    -0.418887
      two     0.612556
Name: data1, dtype: float64

In [17]:
data.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one     0.349206
b     one     1.629293
      two    -0.434777
Name: data2, dtype: float64

In [19]:
data.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
b     one     1
      two     2
dtype: int64

## ดูข้อมูลของแต่ละกลุ่ม

In [24]:
for name, group in data.groupby('key1'):
    print('Group Name : %s' %name)
    print(group)


Group Name : a
      data1     data2 key1 key2
0  0.453406 -0.112818    a  one
4 -0.368687  0.811231    a  one
Group Name : b
      data1     data2 key1 key2
1  0.167646 -0.868377    b  two
2 -0.418887  1.629293    b  one
3  1.057466 -0.001176    b  two


### ในกรณีถ้าการจัดกลุ่มเป็นแบบ multiple key นั้น

การ iterate ข้อมูลจะ return tuple ของ key ออกมา กับข้อมูล ดังนี้

In [26]:
for (key1, key2), group in data.groupby(['key1','key2']):
    print((key1, key2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0  0.453406 -0.112818    a  one
4 -0.368687  0.811231    a  one
('b', 'one')
      data1     data2 key1 key2
2 -0.418887  1.629293    b  one
('b', 'two')
      data1     data2 key1 key2
1  0.167646 -0.868377    b  two
3  1.057466 -0.001176    b  two


### โดยปกติการ groupby จะทำในแต่ละ row (axis=0)

แต่เราสามารถเปลี่ยนเป็น column ได้ (axis=1)

In [28]:
data

,data1,data2,key1,key2
0,0.453406,-0.112818,a,one
1,0.167646,-0.868377,b,two
2,-0.418887,1.629293,b,one
3,1.057466,-0.001176,b,two
4,-0.368687,0.811231,a,one


### ทำการ group ข้อมูลด้วยชนิดของข้อมูล

In [27]:
data.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [30]:
grouped = data.groupby(data.dtypes, axis=1)
grouped

In [31]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.453406 -0.112818
1  0.167646 -0.868377
2 -0.418887  1.629293
3  1.057466 -0.001176
4 -0.368687  0.811231
object
  key1 key2
0    a  one
1    b  two
2    b  one
3    b  two
4    a  one


## Data Aggregation

ซึ่งมาจาก groupby นั่นเอง มี function ต่าง ๆ มากมาย เช่น count, sum, mean, median, str, var, min, max, prod, first และ last

In [32]:
data

,data1,data2,key1,key2
0,0.453406,-0.112818,a,one
1,0.167646,-0.868377,b,two
2,-0.418887,1.629293,b,one
3,1.057466,-0.001176,b,two
4,-0.368687,0.811231,a,one


In [33]:
grouped = data.groupby('key1')
grouped

In [41]:
grouped['data1'].quantile(0.9)

key1
a    0.371197
b    0.879502
Name: data1, dtype: float64

In [37]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.822094,0.924049
b,1.476353,2.497670


### สามารถใช้ function describe() สำหรับดูข้อมูลเชิงสถิติได้เช่นกัน

In [42]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      2.0  0.042359  0.581308 -0.368687 -0.163164  0.042359  0.247883   
b      3.0  0.268742  0.743350 -0.418887 -0.125620  0.167646  0.612556   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.453406   2.0  0.349206  0.653401 -0.112818  0.118194  0.349206   
b     1.057466   3.0  0.253247  1.268124 -0.868377 -0.434777 -0.001176   

                          
           75%       max  
key1                      
a     0.580219  0.811231  
b     0.814058  1.629293

In [43]:
grouped['data1'].describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,2.0,0.042359,0.581308,-0.368687,-0.163164,0.042359,0.247883,0.453406
b,3.0,0.268742,0.743350,-0.418887,-0.125620,0.167646,0.612556,1.057466


In [44]:
grouped['data2'].describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,2.0,0.349206,0.653401,-0.112818,0.118194,0.349206,0.580219,0.811231
b,3.0,0.253247,1.268124,-0.868377,-0.434777,-0.001176,0.814058,1.629293


## Cross-Tabulations: Crosstab

ใน Pandas library นั้นเตรียม function crosstab() ให้ใช้งาน ซึ่งสะดวกกว่า pivot table อย่างมาก

In [50]:
data = pd.read_csv('demo_data/data06.csv')
data.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [54]:
# Crosstab with multiple index
pd.crosstab([data.time, data.day], data.smoker)

smoker       No  Yes
time   day          
Dinner Fri    3    9
       Sat   45   42
       Sun   57   19
       Thur   1    0
Lunch  Fri    1    6
       Thur  44   17

In [55]:
pd.crosstab([data.time, data.day], data.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [58]:
pd.crosstab([data.time], [data.smoker, data.day], margins=True)

smoker  No              Yes               All
day    Fri Sat Sun Thur Fri Sat Sun Thur     
time                                         
Dinner   3  45  57    1   9  42  19    0  176
Lunch    1   0   0   44   6   0   0   17   68
All      4  45  57   45  15  42  19   17  244